<a href="https://colab.research.google.com/github/harshitadd/DP-NLP/blob/main/LSTMNewsClassificationDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchcsprng==0.1.3+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
!pip install opacus --quiet

     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0+cu92 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [4]:
import zipfile
import urllib.request
import os
import torch
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm
import pandas as pd
from torch.utils.data import Dataset
from pathlib import Path
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
from opacus import PrivacyEngine
from torch.utils.data import TensorDataset

In [6]:
DATA_DIR = "/content/"

In [7]:
def download_and_extract(data_dir):
    print("Extracting Train zip...")
    filename = "train.csv.zip"
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

    print("Extracting Test zip...")
    filename = "test.csv.zip"
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

download_and_extract(DATA_DIR)

Extracting Train zip...
Completed!
Extracting Test zip...
Completed!


In [8]:
train_path =  '/content/train.csv'
dev_path = '/content/test.csv'

df_train = pd.read_csv(train_path)[:2000]
df_test = pd.read_csv(dev_path)[:400]
df_train = df_train.drop('Title', axis = 1)
df_test = df_test.drop('Title', axis = 1)

In [9]:
df = pd.concat([df_train, df_test])

In [10]:
class CharByteEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.start_token = "<s>"
        self.end_token = "</s>"
        self.pad_token = "<pad>"
        self.start_idx = 256
        self.end_idx = 257
        self.pad_idx = 258

    def forward(self, s: str, pad_to=0) -> torch.LongTensor:

        encoded = s.encode()
        n_pad = pad_to - len(encoded) if pad_to > len(encoded) else 0
        return torch.LongTensor(
            [self.start_idx]
            + [c for c in encoded]  # noqa
            + [self.end_idx]
            + [self.pad_idx for _ in range(n_pad)]
        )

    def decode(self, char_ids_tensor: torch.LongTensor) -> str:
        char_ids = char_ids_tensor.cpu().detach().tolist()

        out = []
        buf = []
        for c in char_ids:
            if c < 256:
                buf.append(c)
            else:
                if buf:
                    out.append(bytes(buf).decode())
                    buf = []
                if c == self.start_idx:
                    out.append(self.start_token)
                elif c == self.end_idx:
                    out.append(self.end_token)
                elif c == self.pad_idx:
                    out.append(self.pad_token)

        if buf:  # in case some are left
            out.append(bytes(buf).decode())
        return "".join(out)

    def __len__(self):
        return 259

In [11]:
class NewsClassification(Dataset):
    def __init__(self , df): # df with the news description and label 
        self.labels = df['Class Index']
        self.data = df['Description']
        self.encoder = CharByteEncoder()
        self.processed = self.process_samples()

    def __getitem__(self, i):
      return self.processed[i]
    
    def process_samples(self):
      processed = []
      for d, l in zip(self.data, self.labels):
        processed.append((self.encoder(d.strip()), torch.tensor(l).long()))
      return processed

    def __len__(self):
        return len(self.data)
      

VOCAB_SIZE = 256 + 3  # 256 alternatives in one byte, plus 3 special characters.

In [ ]:
  # (xx, yy) = zip(*batch)
    # x_lens = [len(x) for x in xx]
    # print(len(batch))
    # print(x_lens)
    # xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    # # x = pack_padded_sequence(xx_pad, x_lens, batch_first=False, enforce_sorted=True)
    # yy_pad = torch.stack([elem[1] for elem in batch]).long()
    # return xx_pad, yy_pad

      # x_lens = [len(x) for x in xx]

    # print(np.shape(x), print(np.shape(y)))
    # return x, y

    # data = [item[0] for item in batch]
    # data = pack_sequence(data, enforce_sorted=False)
    # targets = torch.stack([item[1] for item in batch]).long()
    # return data, targets

      # max_len = max(map(lambda x: x[0].shape[self.dim], batch))
      # # pad according to max_len
      # batch = map(lambda (x, y):(pad_tensor(x, pad=max_len, dim=self.dim), y), batch)
      # # stack all
      # xs = torch.stack(map(lambda x: x[0], batch), dim=0)
      # ys = torch.LongTensor(map(lambda x: x[1], batch))
      # return xs, ys

    # batch_size = len(batch)

    # sample_batch, target_batch = [], []
    # for sample, target in batch:

    #     sample_batch.append(sample)
    #     target_batch.append(target)

    # padded_batch = pad_sequence(sample_batch, batch_first=True, padding_value = padding_idx)
    # padded_to = list(padded_batch.size())[1]
    # padded_batch = padded_batch.reshape(len(sample_batch), padded_to, 1)        
    
    # return padded_batch, torch.cat(target_batch, dim=0).reshape(len(sample_batch))

In [12]:
from torch.nn.utils.rnn import pad_sequence
def padded_collate(batch, padding_idx=0):

    xx = pad_sequence([elem[0] for elem in batch], batch_first=True, padding_value=padding_idx)
    y = torch.stack([elem[1] for elem in batch]).long()
    return xx, y

In [13]:
secure_rng = False
generator = None 
train_split = 0.8
test_every = 5
batch_size = 32

In [14]:
ds = NewsClassification(df)
train_len = int(train_split * len(ds))
test_len = len(ds) - train_len

In [15]:
print(f"{train_len} samples for training, {test_len} for testing")

1920 samples for training, 480 for testing


In [16]:
train_ds, test_ds = torch.utils.data.random_split(ds, [train_len, test_len], generator=None)

In [17]:
from torch.nn.utils.rnn import pack_padded_sequence
sample_rate = batch_size / len(train_ds)

train_loader = DataLoader(
    train_ds,
    num_workers=0,
    pin_memory=True,
    generator=generator,
    batch_sampler=UniformWithReplacementSampler(
        num_samples=len(train_ds),
        sample_rate=sample_rate,
        generator=generator,
    ),
    collate_fn=padded_collate,
)

test_loader = DataLoader(
    test_ds,
    batch_size=2 * batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=padded_collate,
)

In [18]:
from opacus.layers import DPLSTM

class CharNNClassifier(nn.Module):
    def __init__(
        self,
        embedding_size,
        hidden_size,
        output_size,
        num_lstm_layers=1,
        bidirectional=False,
        vocab_size=VOCAB_SIZE,
    ):
        super().__init__()

        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = DPLSTM(
            embedding_size,
            hidden_size,
            num_layers=num_lstm_layers,
            bidirectional=bidirectional,
            batch_first=True,
        )
        self.out_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)  # -> [B, T, D]
        x, _ = self.lstm(x, hidden)  # -> [B, T, H]
        x = x[:, -1, :]  # -> [B, H]
        x = self.out_layer(x)  # -> [B, C]
        return x

In [19]:
# Set the device to run on a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define classifier parameters
embedding_size = 64
hidden_size = 128  # Number of neurons in hidden layer after LSTM
n_lstm_layers = 1
bidirectional_lstm = False

model = CharNNClassifier(
    embedding_size,
    hidden_size,
    len(ds.labels),
    n_lstm_layers,
    bidirectional_lstm,
).to(device)

In [20]:
# Training hyper-parameters
epochs = 20
learning_rate = 2.0

# Privacy engine hyper-parameters
max_per_sample_grad_norm = 1.5
delta = 8e-5
epsilon = 12.0

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine(
    model,
    sample_rate=sample_rate,
    max_grad_norm=max_per_sample_grad_norm,
    target_delta=delta,
    target_epsilon=epsilon,
    epochs=epochs,
    secure_rng=secure_rng,
)
privacy_engine.attach(optimizer)

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:523: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


In [23]:
from statistics import mean

def train(model, criterion, optimizer, train_loader, epoch, device="cuda:0"):
    accs = []
    losses = []
    counter = 0 
    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = y.to(device)


        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        preds = logits.argmax(-1)
        n_correct = float(preds.eq(y).sum())
        batch_accuracy = n_correct / len(y)

        accs.append(batch_accuracy)
        losses.append(float(loss))

        # print('Tensor Shape', (np.shape(x)[0], np.shape(y)[0]))
        # counter += 1 
        # printstr = (f"\t Epoch {epoch}. Batch Failed")
    
    printstr = (f"\t Epoch {epoch}. Accuracy: {mean(accs):.6f} | Loss: {mean(losses):.6f}")
        
    try:
        privacy_engine = optimizer.privacy_engine
        epsilon, best_alpha = privacy_engine.get_privacy_spent()
        printstr += f" | (ε = {epsilon:.2f}, δ = {privacy_engine.target_delta}) for α = {best_alpha}"
    except :
        pass

    print(printstr)
    return counter 


def test(model, test_loader, privacy_engine, device="cuda:0"):
    accs = []
    with torch.no_grad():
        for x, y in tqdm(test_loader):
            x = x.to(device)
            y = y.to(device)
            preds = model(x).argmax(-1)
            n_correct = float(preds.eq(y).sum())
            batch_accuracy = n_correct / len(y)

            accs.append(batch_accuracy)
    printstr = "\n----------------------------\n" f"Test Accuracy: {mean(accs):.6f}"
    if privacy_engine:
        epsilon, best_alpha = privacy_engine.get_privacy_spent()
        printstr += f" (ε = {epsilon:.2f}, δ = {privacy_engine.target_delta}) for α = {best_alpha}"
    print(printstr + "\n----------------------------\n")
    return

In [24]:
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import pack_padded_sequence

In [27]:
from tqdm import tqdm

print("Train stats: \n")
for epoch in tqdm(range(epochs)):
    train(model, criterion, optimizer, train_loader, epoch, device=device)
    if test_every:
        if epoch % test_every == 0:  test(model, test_loader, privacy_engine, device=device)

  0%|          | 0/60 [00:00<?, ?it/s]

Train stats: 




100%|██████████| 60/60 [00:34<00:00,  1.73it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	 Epoch 0. Accuracy: 0.282511 | Loss: 2.886188 | (ε = 5.09, δ = 8e-05) for α = 3.3



  0%|          | 0/60 [00:00<?, ?it/s]


----------------------------
Test Accuracy: 0.230469 (ε = 5.09, δ = 8e-05) for α = 3.3
----------------------------



  0%|          | 0/60 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
test(model, test_loader, privacy_engine, device=device)

WITHOUT PRIVACY

In [28]:
model_nodp = CharNNClassifier(
    embedding_size,
    hidden_size,
    len(ds.labels),
    n_lstm_layers,
    bidirectional_lstm,
).to(device)


optimizer_nodp = torch.optim.SGD(model_nodp.parameters(), lr=0.5)

In [29]:
for epoch in tqdm(range(10)):
    train(model_nodp, criterion, optimizer_nodp, train_loader, epoch, device=device)
    if test_every:
        if epoch % test_every == 0:
            test(model_nodp, test_loader, None, device=device)

test(model_nodp, test_loader, None, device=device)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  2%|▏         | 1/60 [00:00<00:16,  3.57it/s]

  3%|▎         | 2/60 [00:00<00:15,  3.69it/s]

  5%|▌         | 3/60 [00:00<00:15,  3.79it/s]

  7%|▋         | 4/60 [00:01<00:15,  3.60it/s]

  8%|▊         | 5/60 [00:01<00:14,  3.73it/s]

 10%|█         | 6/60 [00:01<00:17,  3.01it/s]

 12%|█▏        | 7/60 [00:02<00:16,  3.18it/s]

 13%|█▎        | 8/60 [00:02<00:14,  3.63it/s]

 15%|█▌        | 9/60 [00:02<00:13,  3.77it/s]

 17%|█▋        | 10/60 [00:02<00:16,  3.02it/s]

 18%|█▊        | 11/60 [00:03<00:17,  2.77it/s]

 20%|██        | 12/60 [00:03<00:15,  3.12it/s]

 22%|██▏       | 13/60 [00:03<00:14,  3.31it/s]

 23%|██▎       | 14/60 [00:04<00:13,  3.47it/s]

 25%|██▌       | 15/60 [00:04<00:13,  3.42it/s]

 27%|██▋       | 16/60 [00:04<00:12,  3.59it/s]

 28%|██▊       | 17/60 [00:04<00:11,  3.78it/s]

 30%|███       | 18/60 [00:05<00:13,  3.04it/s]

 32%|███▏      | 19/60 [00:05<00:12,  3.31it/s

	 Epoch 0. Accuracy: 0.279363 | Loss: 2.113450




 25%|██▌       | 2/8 [00:00<00:01,  5.33it/s]

 38%|███▊      | 3/8 [00:00<00:00,  5.45it/s]

 50%|█████     | 4/8 [00:00<00:00,  6.09it/s]

 62%|██████▎   | 5/8 [00:00<00:00,  5.41it/s]

 75%|███████▌  | 6/8 [00:01<00:00,  5.99it/s]

 88%|████████▊ | 7/8 [00:01<00:00,  6.33it/s]

100%|██████████| 8/8 [00:01<00:00,  5.83it/s]

 10%|█         | 1/10 [00:19<02:57, 19.75s/it]

  0%|          | 0/60 [00:00<?, ?it/s]


----------------------------
Test Accuracy: 0.373047
----------------------------





  2%|▏         | 1/60 [00:00<00:14,  4.04it/s]

  3%|▎         | 2/60 [00:00<00:15,  3.76it/s]

  5%|▌         | 3/60 [00:00<00:14,  3.92it/s]

  7%|▋         | 4/60 [00:01<00:14,  3.74it/s]

  8%|▊         | 5/60 [00:01<00:14,  3.73it/s]

 10%|█         | 6/60 [00:01<00:14,  3.62it/s]

 12%|█▏        | 7/60 [00:01<00:13,  3.89it/s]

 13%|█▎        | 8/60 [00:02<00:14,  3.59it/s]

 15%|█▌        | 9/60 [00:02<00:13,  3.84it/s]

 17%|█▋        | 10/60 [00:02<00:13,  3.73it/s]

 18%|█▊        | 11/60 [00:02<00:13,  3.75it/s]

 20%|██        | 12/60 [00:03<00:14,  3.35it/s]

 22%|██▏       | 13/60 [00:03<00:14,  3.31it/s]

 23%|██▎       | 14/60 [00:03<00:14,  3.18it/s]

 25%|██▌       | 15/60 [00:04<00:12,  3.53it/s]

 27%|██▋       | 16/60 [00:04<00:15,  2.90it/s]

 28%|██▊       | 17/60 [00:04<00:13,  3.18it/s]

 30%|███       | 18/60 [00:05<00:12,  3.25it/s]

 32%|███▏      | 19/60 [00:05<00:11,  3.50it/s]

 33%|███▎      | 20/60 [00:05<00:11,  3.46it/s]

 35%|███▌      | 21/60 [00:

...

	 Epoch 8. Accuracy: 0.319723 | Loss: 1.393438




  2%|▏         | 1/60 [00:00<00:31,  1.89it/s]

  3%|▎         | 2/60 [00:00<00:25,  2.31it/s]

  5%|▌         | 3/60 [00:01<00:26,  2.17it/s]

  7%|▋         | 4/60 [00:01<00:22,  2.52it/s]

  8%|▊         | 5/60 [00:01<00:19,  2.89it/s]

 10%|█         | 6/60 [00:02<00:17,  3.01it/s]

 12%|█▏        | 7/60 [00:02<00:16,  3.18it/s]

 13%|█▎        | 8/60 [00:02<00:15,  3.30it/s]

 15%|█▌        | 9/60 [00:03<00:17,  2.85it/s]

 17%|█▋        | 10/60 [00:03<00:18,  2.63it/s]

 18%|█▊        | 11/60 [00:03<00:17,  2.80it/s]

 20%|██        | 12/60 [00:04<00:16,  2.99it/s]

 22%|██▏       | 13/60 [00:04<00:15,  3.03it/s]

 23%|██▎       | 14/60 [00:04<00:14,  3.22it/s]

 25%|██▌       | 15/60 [00:04<00:13,  3.43it/s]

 27%|██▋       | 16/60 [00:05<00:12,  3.58it/s]

 28%|██▊       | 17/60 [00:05<00:11,  3.78it/s]

 30%|███       | 18/60 [00:05<00:11,  3.76it/s]

 32%|███▏      | 19/60 [00:05<00:11,  3.61it/s]

 33%|███▎      | 20/60 [00:06<00:13,  3.00it/s]

 35%|███▌      | 21/60 [00:

	 Epoch 9. Accuracy: 0.329200 | Loss: 1.388969



100%|██████████| 8/8 [00:01<00:00,  5.86it/s]


----------------------------
Test Accuracy: 0.242188
----------------------------

